# World and Transformations/Poses in the Solution Building Library

This example notebook demonstrates how to interact with the scene/world using the Intrinsic Solution Building Library.

<div class="alert alert-info">

**Important**

This notebook requires a running Flowstate solution to connect to. To start a solution:

1. Navigate to [portal.intrinsic.ai](https://portal.intrinsic.ai/) and sign in
   using your registered Flowstate account.

1. Do **one** of the following:
    - Create a new solution:
        1. Click "Create new solution" and choose "From an example".
        1. Select `pick_and_place:pick_and_place_module2`
        1. Click "Create".
    - Or open an existing solution that was created from the `pick_and_place:pick_and_place_module2` example:
        1. Hover over the solution in the list.
        1. Click "Open solution" or "Start solution".

1. Recommended: Keep the browser tab with the Flowstate solution editor open to watch the effect of notebook actions such as running a skill. You can simultaneously interact with the solution through the web UI and the notebook.

</div>

## Connect to solution

Let's start with the typical preamble:

- Import the relevant modules.
- Connect to the deployed solution.
- Define some shortcut variables for convenience.

In [ ]:
import math

from intrinsic.math.python import data_types
from intrinsic.solutions import deployments
from intrinsic.solutions import behavior_tree as bt

solution = deployments.connect_to_selected_solution()

executive = solution.executive
resources = solution.resources
skills = solution.skills
world = solution.world

## Interacting with the world

`solution.world` provides programmatic access to the **Belief** world which you can see in the [workcell designer](https://developers.intrinsic.ai/guides/workcell_design/workcell_overview) of the Flowstate solution editor.

From Python, you can query the belief world state, e.g., for object poses and also do a limited amount of editing. E.g., you can create frames and move objects. In the following we will show a few examples. Note that this is the same Python API that is also used for [accessing the belief world from Python skills](https://developers.intrinsic.ai/guides/skill_authoring/access_belief_world).


### Print the world to get an overview

The `print()` function prints the world as a tree structure. In the output, the double arrows (=>) symbolize child objects, the normal arrows (->) symbolize child frames.

In [ ]:
print(world)

### Access an object

World objects can be accessed with the notation `world.<object_name>` or via `world.get_object()` (if the object's name is marked as a global alias in the world).

In [ ]:
building_block = world.building_block0
print(building_block)
print("name:", building_block.name)
print("parent_name:", building_block.parent_name)

# Access the same box by name using the get_object method.
building_block_2 = world.get_object("building_block0")
print(building_block_2)


### Access a frame

Frames can either be global (=attached to the world root) or local (=attached to an object).

Global frames can be accessed with the notation `world.<frame_name>` or via `world.get_frame()`:

In [ ]:
grasp_frame = world.grasp_frame
print(grasp_frame)
print("name:", grasp_frame.name)
print("object_name:", grasp_frame.object_name)

# Access the same frame by frame name and object name using the get_frame method.
grasp_frame = world.get_frame("grasp_frame", "root")
print(grasp_frame)

Local frames can be accessed with the notation `world.<object_name>.<frame_name>` or via `world.get_frame()`. Note that all frames under one object always need to have unique names but the same frame name can be reused under different objects:

In [ ]:
gripper_tool_frame = world.picobot_gripper.tool_frame
print(gripper_tool_frame)
print("name:", gripper_tool_frame.name)
print("object_name:", gripper_tool_frame.object_name)

# Access the same frame by frame name and object name using the get_frame method.
gripper_tool_frame = world.get_frame("tool_frame", "picobot_gripper")
print(gripper_tool_frame)

### Access a kinematic objects

Kinematic objects are objects which have movable joints and thus have more properties and extendend functionality. Common examples are robots and grippers.

Here are a few common operations:

In [ ]:
robot = world.robot
print(robot)

# Access the same robot object by object name using the get_kinematic_object method.
robot = world.get_kinematic_object("robot")

# Get the current joint positions of the robot in the belief world.
print("Current position:", robot.joint_positions)

# Get the robot joint configuration which is saved under the name "home".
print('Saved "home" position:', robot.joint_configurations.home.joint_position)

### Exploring the world

Here are a few more examples on how the world can be explored:

In [ ]:
print("All objects in the world:")
print(world.list_objects(), "\n")

print("The names of all objects in the world:")
print(world.list_object_names(), "\n")

print("All child frames of the gripper object:")
print(world.picobot_gripper.frames, "\n")

print("The names of all frames of the gripper object:")
print(world.picobot_gripper.frame_names)

### Access transformations

It is possible to access transformations between objects and/or frames:

In [ ]:
# Access the transform from the parent to the building block object.
print(world.building_block0.parent_t_this)

# Get the transform between any combination of frame and/or object.
robot_t_block = world.get_transform(world.robot, world.building_block0)
print(robot_t_block)

You can also manipulate the transforms in the belief world. This results in moving the objects/frames:

In [ ]:
from intrinsic.math.python import data_types

# Move the 'building_block0' object to 1 cm above the 'target_right' frame.
# - The first three arguments specify that the transform between the 'target_right' frame and
#   the 'building_block0' object (="target_right_t_building_block") should become (0, 0, 0.01).
# - The fourth argument specifies that this should be achieved by modifying the transform between
#   the 'building_block0' object and its immediate parent (=the world root).
world.update_transform(
    world.target_right,
    world.building_block0,
    data_types.Pose3(translation=[0, 0, 0.01]),
    world.building_block0,
)

After executing the `world.update_transform()` call you should be able to observe in the Flowstate solution editor that the `building_block` object has moved.

### Reparent objects
Objects can be re-attached to another parent in the worlds scene tree.

In [ ]:
# Reparent the connector on the box
print(world.building_block0.parent_name)
world.reparent_object(world.building_block0, new_parent=world.building_block_tray)
print(world.building_block0.parent_name)

Now the `building_block0` object can be found in the tree under `building_block_tray`.

In [ ]:
print(world)

## Poses

When working with skills and the belief world you will often have to deal with poses and rotations. For this purpose, the Solution Building Library uses the `intrinsic.math.python.data_types` module from the Intrinsic SDK. In the following we show a few examples of how to use the `Pose3` type and its friends.

There are three ways to create a pose consisting of translation and rotation. Firstly, it can be created from a 7-element vector containing translation and rotation (as a quarternion), [tx, ty, tz, qx, qy, qz, qw]:



In [ ]:
from intrinsic.math.python import data_types

pose = data_types.Pose3.from_vec7([0.1, 0.2, 0.3, 0.27060, 0.65328, -0.27060, 0.65328])
print("Vec7 representation (px,py,pz,x,y,z,w):", pose.vec7)
print(pose)


The same pose can be constructed by creating the quarternion explicitly and specifying translation and orientation separately:


In [ ]:
pose = data_types.Pose3(
    rotation=data_types.Rotation3(
        data_types.Quaternion([0.27060, 0.65328, -0.27060, 0.65328])
    ),
    translation=[0.1, 0.2, 0.3],
)
print(pose)


If you don't want to specify quarternions, you can also use Euler angles:



In [ ]:
pose = data_types.Pose3(
    rotation=data_types.Rotation3.from_euler_angles(rpy_degrees=[45, 90, 0]),
    translation=[0.1, 0.2, 0.3],
)
print(pose)


As you can see, all three ways of specifying a pose lead to the same pose.

As you saw in a previous section you can also query poses from the belief world, e.g.:

In [ ]:
root_t_block = world.get_transform(world.root, world.building_block0)
print(root_t_block)

For pose computations the `Pose3` class supports the `*` operator for pose concatenation and has the `inverse()` method. Usage example:

In [ ]:
camera_t_block = world.get_transform(world.wrist_camera, world.building_block0)
tray_t_camera = world.get_transform(world.building_block_tray, world.wrist_camera)
camera_t_tray = world.get_transform(world.wrist_camera, world.building_block_tray)

tray_t_block_1 = tray_t_camera * camera_t_block
tray_t_block_2 = camera_t_tray.inverse() * camera_t_block

# Should print the same pose three times
print(world.get_transform(world.building_block_tray, world.building_block0))
print(tray_t_block_1)
print(tray_t_block_2)

## Next steps

Take a look at the following example notebooks to learn:

- How to [parameterize skill instances](004_skills.ipynb).
- How to create behavior trees with control flow nodes such as [sequences](005_sequence.ipynb), [loops and branches](006_loop_and_branch.ipynb) or [retries](007_retry.ipynb).